# Classifier-Free Guidance.

**Description:** Training the Flow Matching model with Classifier-Free Guidance. 

In [1]:
import os
import sys
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import anndata as ad
import matplotlib.pyplot as plt
sys.path.append("../../")
from utils.autoencoder_utils import NB_Autoencoder
from utils.classifier_free_guidance import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Configuration & Data Loading

In [4]:
# Paths
input_file_path = "/dtu/blackhole/06/213542/paperdata/pbmc3k_train_with_latent.h5ad"
flow_model_save_path = "/dtu/blackhole/06/213542/paperdata/lib_size_flow_model.pt"

os.makedirs(os.path.dirname(flow_model_save_path), exist_ok=True)

# Hyperparameters
batch_size = 256
num_epochs = 400
learning_rate = 5e-4
latent_dim = 50
p_uncond = 0.1   # Classifier-free guidance dropout probability

# Load Data
adata = ad.read_h5ad(input_file_path)
latent = adata.obsm["X_latent"]
latent_tensor = torch.tensor(latent, dtype=torch.float32, device=device)

# Library Sizes
if "total_counts" in adata.obs:
    lib_sizes = adata.obs["total_counts"].values
else:
    lib_sizes = np.array(adata.X.sum(1)).flatten()

log_lib_sizes = np.log1p(lib_sizes)
log_lib_tensor = torch.tensor(log_lib_sizes, dtype=torch.float32, device=device).unsqueeze(1)

# Stats for normalization (needed for training context if we normalize, but mostly for sampling later)
lib_min, lib_max = log_lib_tensor.min(), log_lib_tensor.max()
lib_mean, lib_std = log_lib_tensor.mean(), log_lib_tensor.std()

# Cell Types
cell_types = adata.obs["cell_type"].astype(str).values
unique_types, inverse_idx = np.unique(cell_types, return_inverse=True)
num_cell_types = len(unique_types)
cell_type_idx = torch.tensor(inverse_idx, dtype=torch.long, device=device)

print(f"Data Shape: {latent.shape}")
print(f"Library Size: Min={lib_min:.2f}, Max={lib_max:.2f}, Mean={lib_mean:.2f}")
print(f"Cell Types: {unique_types}")

Data Shape: (2110, 50)
Library Size: Min=6.49, Max=7.86, Mean=7.42
Cell Types: ['B cells' 'CD14+ Monocytes' 'CD4 T cells' 'CD8 T cells' 'Dendritic cells'
 'FCGR3A+ Monocytes' 'Megakaryocytes' 'NK cells']


## Training Loop

In [5]:
# Initialisation
emp_dist = EmpiricalDistribution(latent_tensor)
latent_dim = latent_tensor.shape[1]

# Initialize models
conditioner = CellTypeConditioner(n_types=num_cell_types, latent_dim=latent_dim).to(device)
vf_model = NeuralVectorField(latent_dim=latent_dim).to(device)

# Optimize both model and conditioner
optimizer = torch.optim.AdamW(list(vf_model.parameters()) + list(conditioner.parameters()), lr=learning_rate)

epochs_list = []
loss_list = []

print("Starting CFG Training...")

for epoch in range(num_epochs):
    # 1. Sample Indices and Data
    indices = torch.randint(0, latent_tensor.shape[0], (batch_size,))
    z = latent_tensor[indices].to(device)          # Target (Data)
    c_idx = cell_type_idx[indices].to(device)      # Condition (Cell Type)
    l = log_lib_tensor[indices].to(device)         # Condition (Library Size)

    # 2. Sample Noise and Time
    x = torch.randn(batch_size, latent_dim, device=device)
    t = torch.rand(batch_size, 1, device=device)
    
    # 3. Compute Flow Target (Noise -> Data)
    u_target = z - x 
    
    # Normalize target
    u_mean = u_target.mean(dim=0, keepdim=True)
    u_std = u_target.std(dim=0, keepdim=True) + 1e-6
    u_target_norm = (u_target - u_mean) / u_std
    
    # 4. CFG: Condition masking
    c_emb = conditioner(c_idx)
    
    # Create mask (1 = drop condition, 0 = keep condition)
    mask = (torch.rand(batch_size, 1, device=device) < p_uncond).float()
    
    # Apply Mask: Replace dropped conditions with null_cond
    c_input = mask * vf_model.null_cond.expand(batch_size, -1) + (1 - mask) * c_emb
    
    # 5. Forward pass
    v_pred = vf_model(x, c_input, t, l)
    
    # 6. Loss
    loss = F.mse_loss(v_pred, u_target_norm)
    
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(vf_model.parameters(), 1.0)
    optimizer.step()
    
    epochs_list.append(epoch)
    loss_list.append(loss.item())
    
    if epoch % 50 == 0:
        print(f"[{epoch}] Loss: {loss.item():.6f}")

# Save Models
torch.save({
    'vf_state': vf_model.state_dict(),
    'cond_state': conditioner.state_dict()
}, flow_model_save_path)
print(f"Saved models to {flow_model_save_path}")

Starting CFG Training...
[0] Loss: 1.435534
[50] Loss: 0.866035
[100] Loss: 0.762702
[150] Loss: 0.739563
[200] Loss: 0.733395
[250] Loss: 0.729226
[300] Loss: 0.729674
[350] Loss: 0.729088
Saved models to /dtu/blackhole/06/213542/paperdata/lib_size_flow_model.pt
